## Do it in Google colab as this notebook requires GPU

In [1]:
!pip install langchain_community neo4j langchain langchain_groq sentence-transformers langchain_huggingface langchain_nvidia_ai_endpoints

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: lan

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-1ejnfrso/unsloth_df9b8bc70f0f43ebb9e0620cd1da74ad
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-1ejnfrso/unsloth_df9b8bc70f0f43ebb9e0620cd1da74ad
  Resolved https://github.com/unslothai/unsloth.git to commit 79db7130734056928fc25e38a0fde64b72d5615e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 7.4 MB/s eta 0:00:00
   

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 101.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.chains import GraphCypherQAChain
from google.colab import userdata

In [2]:
neo4j_url = userdata.get("NEO4J_CONNECTION_URL")
neo4j_user = userdata.get("NEO4J_USER")
neo4j_password = userdata.get("NEO4J_PASSWORD")
graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)

<ipython-input-2-faf1b39b523e>:4: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)


In [3]:
groq_api_key = userdata.get("GROQ_API_KEY")
cypher_model = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant", groq_api_key = groq_api_key)
# qa_model = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile", groq_api_key = groq_api_key)
qa_model = ChatGroq(temperature=0, model_name="gemma2-9b-it", groq_api_key = groq_api_key)
# qa_model = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)

In [6]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # to restricts its context window, most of model support larger, but make sure to use it based on your need since it will consumes more compute and VRAM
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested 8, 16, 32, 64, 128, higher ranks can store more information but increase the computational and memory cost of LoRA.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # rule of thumb, double the r or equal. alpha is a scaling factor for updates
    lora_dropout = 0, # Supports any, but = 0 is optimized. Probability of zeroing out elements in low-rank matrices for regularization.
    random_state = 3407,
    use_rslora = True,  # has been proven to work better ( https://arxiv.org/pdf/2312.03732 )
)

Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = f"Convert text to cypher query based on this schema: {graph.schema}" # examples["instructions"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [9]:
df=pd.read_excel('/content/TextCypherTrainingData.xlsx')
print(df.shape)
df = df[['question','cypher']]
df.rename(columns={'question': 'input','cypher':'output'}, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

(211, 6)


,input,output
0,Which clubs have the most players with an over...,MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE ...
1,Which clubs have the most players with a value...,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[..."
2,Which clubs have the most players with a best ...,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[..."
3,What is the average wage of all players with a...,MATCH (p:Player) WHERE p.best_overall > 85 MAT...
4,Which players have a best overall rating above...,MATCH (p:Player) WHERE p.best_overall > 85 RET...


In [10]:
neo4j_url = userdata.get("NEO4J_CONNECTION_URL")
neo4j_user = userdata.get("NEO4J_USER")
neo4j_password = userdata.get("NEO4J_PASSWORD")
graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)

In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset

Parameter 'function'=<function formatting_prompts_func at 0x7ccb9ba29870> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/211 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 211
})

In [12]:
dataset[0]

{'input': 'Which clubs have the most players with an overall rating above 85?',
 'output': 'MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE p.overall_rating > 85 RETURN c.name AS club_name, COUNT(p) AS player_count ORDER BY player_count DESC',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert text to cypher query based on this schema: Node properties:\n- **Player**\n  - `name`: STRING Example: "Ronaldinho"\n  - `age`: FLOAT Min: 16.0, Max: 44.0\n  - `overall_rating`: FLOAT Min: 47.0, Max: 91.0\n  - `potential`: FLOAT Min: 54.0, Max: 94.0\n  - `best_overall`: FLOAT Min: 49.0, Max: 93.0\n  - `value`: FLOAT Min: 0.0, Max: 1.85E8\n  - `wage`: FLOAT Min: 0.0, Max: 350000.0\n  - `release_clause`: FLOAT Min: 0.0, Max: 3.561E8\n- **Club**\n  - `name`: STRING Example: "Querétaro"\n- **Foot**\n  - `type`: STRING Available options: [\'Right\', \'Left\']\

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs=1,
        learning_rate = 2e-4,  # the rate at which the model updates its parameters during training.
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/211 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 211 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 26
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.154400
2,1.175100
3,1.059800
4,0.895000
5,0.703300
6,0.500300
7,0.359500
8,0.261600
9,0.222300
10,0.149600


In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "A worker who graduated from Simon Fraser University is currently employed at?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert text to cypher query based on this schema: Node properties:
- **Player**
  - `name`: STRING Example: "Ronaldinho"
  - `age`: FLOAT Min: 16.0, Max: 44.0
  - `overall_rating`: FLOAT Min: 47.0, Max: 91.0
  - `potential`: FLOAT Min: 54.0, Max: 94.0
  - `best_overall`: FLOAT Min: 49.0, Max: 93.0
  - `value`: FLOAT Min: 0.0, Max: 1.85E8
  - `wage`: FLOAT Min: 0.0, Max: 350000.0
  - `release_clause`: FLOAT Min: 0.0, Max: 3.561E8
- **Club**
  - `name`: STRING Example: "Querétaro"
- **Foot**
  - `type`: STRING Available options: ['Right', 'Left']
- **Position**
  - `name`: STRING Example: "CAM"
- **Role**
  - `name`: STRING Example: "Central Attacking Midfielder"
- **Performance**
  - `overall_rating`: FLOAT Min: 47.0, Max: 91.0
  - `potential`: FLOAT Min: 54.0, Max: 94.0
  - `best_overall`: FLOAT

In [16]:

model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [18]:
# should have write access
hf_api = '<your-token>'
model.push_to_hub("vishalajairosen/footbal-graph-rag", token = hf_api)
tokenizer.push_to_hub("vishalajairosen/footbal-graph-rag", token = hf_api)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/vishalajairosen/footbal-graph-rag


README.md:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [19]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "vishalajairosen/footbal-graph-rag", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [22]:
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "List players whoose potential is more than 80 and plays as CAM", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
result = tokenizer.batch_decode(outputs)
response = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "").replace("", "")
print(response)

MATCH (p:Player)-[:PLAYS_AS]->(pos:Position {name: 'CAM'}) WHERE p.potential > 80 RETURN p


In [24]:
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

CYPHER_QA_TEMPLATE = """You convert context to a final answer. Understand the question, the context, then generate result.
Here is an example:

Question: Who is the director of Harry Potter 1 and 8?
Context: [{{d.name: Chris Columbus, d.born: 10 September 1958}},{{d.name: David Yates, d.born: 8 October 1963}}]
Helpful Answer: Chris Columbus and David Yates is the director of Harry Potter

Follow this example when generating answers.
Answer in short, don't hallucinate!
Question: {question}
Information: {context}
Helpful Answer:
"""

qa_prompt = ChatPromptTemplate.from_template(CYPHER_QA_TEMPLATE)
output_parser = StrOutputParser()
llm = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile", groq_api_key = groq_api_key)
chain = qa_prompt | llm | output_parser

In [40]:
questions = ['List all the players of FC Barcelona']

def generate_cypher_query(question):
  inputs = tokenizer(
  [
      prompt.format(
          f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  result = tokenizer.batch_decode(outputs)
  cypher_query = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "").replace("", "").replace("{{", "{").replace("}}", "}")
  return cypher_query

for q in questions:
    print("\n", q)
    try:
        cypher_query = generate_cypher_query(q)
        print(cypher_query)
        context = graph.query(cypher_query)
        print('context: ', context)
        result = chain.invoke({"context": context, "question": q})
        print(result)
    except Exception as e:
        # Check if the exception is a CypherSyntaxError and handle it
        if 'Neo.ClientError.Statement.SyntaxError' in str(e):
            print(f"CypherSyntaxError occurred while processing the question: '{q}'. Skipping this question.")
        else:
            print(f"An unexpected error occurred: {e}")



 List all the players of FC Barcelona
MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE c.name = 'FC Barcelona' RETURN p
context:  [{'p': {'name': 'Vitor Roque', 'overall_rating': 76.0, 'potential': 88.0, 'best_overall': 78.0, 'value': 17500000.0, 'age': 18.0, 'release_clause': 39400000.0, 'wage': 44000.0}}, {'p': {'name': 'Pau Cubarsí', 'overall_rating': 64.0, 'potential': 83.0, 'best_overall': 66.0, 'value': 1400000.0, 'age': 16.0, 'release_clause': 3700000.0, 'wage': 1000.0}}, {'p': {'name': 'Balde', 'overall_rating': 81.0, 'potential': 89.0, 'best_overall': 81.0, 'value': 53000000.0, 'age': 19.0, 'release_clause': 119300000.0, 'wage': 70000.0}}, {'p': {'name': 'R. Lewandowski', 'overall_rating': 90.0, 'potential': 90.0, 'best_overall': 90.0, 'value': 58000000.0, 'age': 34.0, 'release_clause': 118900000.0, 'wage': 340000.0}}, {'p': {'name': 'Pedri', 'overall_rating': 86.0, 'potential': 92.0, 'best_overall': 88.0, 'value': 105000000.0, 'age': 20.0, 'release_clause': 236300000.0, 'wage':

### Drawbacks

1. This approach works best with a high-quality LLM model (preferably a paid one) for generating accurate Cypher queries.
2. Better results can be achieved if the dataset used to train the model is large.
3. This is an experimental project; use it as a reference and avoid directly replicating it.